In [ ]:
import numpy as np
import numpy.random as rn
import math as m
import matplotlib.pyplot as plt
from __future__ import division

G = 6.67e-11
R_jup = 7.785e+11
M_jup = 1.898e+27
V_jup = 1.3058e+4
M_sun = 1.989e+30
R_earth = 1.496e+11 #Smallest R for an asteroid

In [ ]:
nbodies = 0
t_intervals = 1000000
t_end = 10000000000
bodieslist = ["Sun", "Jupiter"]
initcond = np.zeros((nbodies,7))
positions = {}
positions["Sun"] = np.array([0,0,0,0,0,0,M_sun]) #x,y,z,vx, vy, vz
positions["Jupiter"] = np.array([R_jup,0,0,0,V_jup,0,M_jup])

print positions

In [ ]:
def f12(Body1,Body2,position_1):
    if Body1 == Body2:
        return np.array([0,0,0])
    pos = position_1-positions[Body2][0:3]
    dist = np.linalg.norm(pos)
    return -G* positions[Body2][-1]*pos/(dist)**3

def force(Body1, position_1):
    F = 0
    for k in ["Jupiter", "Sun"]:
        F = F + f12(Body1, k, position_1)
    return F


def asteroid_init():
    
    for i in range(nbodies):
        theta = 2*m.pi*rn.random()        #Phase of the orbit
        r = rn.uniform(R_earth,R_jup)    #Radius of the asteroids between earth and jupiter orbit
        bodieslist.append("Body "+str(i+1))
        positions[bodieslist[i+2]] = np.array([r*m.cos(theta),r*m.sin(theta),0,(R_jup/r)*V_jup*m.cos(theta+m.pi/2),(R_jup/r)*V_jup*m.sin(theta+m.pi/2),0, 0])
        
    

In [ ]:
def RK4_gravitational(Body, dt):
    
    k1 = np.concatenate((positions[Body][3:6], force(Body, positions[Body][:3]))) * dt
    k2 = np.concatenate((positions[Body][3:6] + k1[3:] * 0.5, force(Body, positions[Body][:3]  + k1[:3] * 0.5))) * dt
    k3 = np.concatenate((positions[Body][3:6] + k2[3:] * 0.5, force(Body, positions[Body][:3]  + k2[:3] * 0.5))) * dt
    k4 = np.concatenate((positions[Body][3:6] + k3[3:], force(Body, positions[Body][:3]  + k3[:3]))) * dt
    positions[Body][:6] = positions[Body][:6] + k1 / 6 + k2 / 3 + k3 / 3 + k4 / 6
    
    return positions[Body]


def integrator():
    asteroid_init()
    dt = t_end / t_intervals
    plt.figure(figsize=(8,6))
    for i in range(t_intervals):
        temporalbodielist = bodieslist
        for j, Body in enumerate(temporalbodielist):
            positions[Body] = RK4_gravitational(Body, dt)
            #Condition to get rid of asteroids flying away
            if positions[Body][0] > 3*R_jup or positions[Body][1] > 3* R_jup or positions[Body][0] < -3*R_jup or positions[Body][1] < -3* R_jup:  #Eliminating from the list asteroids scaping
                bodieslist.remove(Body)
        if (i % 100) == 0:
            plt.plot(positions["Sun"][0], positions["Sun"][1], 'o', c = 'y') 
            plt.plot(positions["Jupiter"][0], positions["Jupiter"][1], 'o', c = 'b') 
            if(nbodies!=0): 
                for k in range(nbodies):
                    plt.plot(positions["Body "+str(k+1)][0], positions["Body "+str(k+1)][1], 'o', c = 'g')  
    plt.show()
    return positions
%timeit print integrator()